In [2]:
import pandas as pd
#import quandl
import nasdaqdatalink
import math
import numpy as np
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pprint
pp = pprint.PrettyPrinter(indent=4)
df = nasdaqdatalink.get("WIKI/GOOGL")
#df = quandl.get("WIKI/GOOGL")
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close'])/df['Adj. Close'] * 100.0
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0

df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]
forecast_col = 'Adj. Close'
df.fillna(-99999, inplace=True)  # fiill in outlier

forecast_out = int(math.ceil(0.01*len(df)))  # Predict with 10% out the data
df['label'] = df[forecast_col].shift(-forecast_out)  
df.dropna(inplace=True)  # We shift the data up and drop the NaN last 10% data in label.
print('df.head():')
pp.pprint(df.head())
X = np.array(df.drop(['label'], axis=1)) # we take out the label data in X.
y = np.array(df['label'])           # we add the label data in y.
X = preprocessing.scale(X)
y = np.array(df['label'])
print('len(X):', len(X), 'len(y):', len(y))

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.2)
# We use Linear Regression
clf = LinearRegression()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print('LinearRegression => accuracy:', accuracy)
# We can use svm.SVR
clf = svm.SVR()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print('svm => accuracy:', accuracy)

df.head():
            Adj. Close    HL_PCT  PCT_change  Adj. Volume      label
Date                                                                
2004-08-19   50.322842  3.712563    0.324968   44659000.0  69.078238
2004-08-20   54.322689  0.710922    7.227007   22834300.0  67.839414
2004-08-23   54.869377  3.729433   -1.227880   18256100.0  68.912727
2004-08-24   52.597363  6.417469   -5.726357   15247300.0  70.668146
2004-08-25   53.164113  1.886792    1.183658    9188600.0  71.219849
len(X): 3389 len(y): 3389
LinearRegression => accuracy: 0.9788671618299348
svm => accuracy: 0.788851989583987
